In [7]:
import lightgbm as lgb
import numpy as np 
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import tensorflow as tf
import warnings

from plotly.subplots import make_subplots
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=FutureWarning)

supply_data = pd.read_csv("supply_chain_data.csv")

ModuleNotFoundError: No module named 'tensorflow'

In [2]:
supply_data.head()

NameError: name 'supply_data' is not defined

In [ ]:
supply_data.shape

In [ ]:
supply_data.columns

In [ ]:
supply_data.info()

In [ ]:
# Check for missing values

missing_values = supply_data.isnull().sum()

# Display columns with missing values and the count of missing values
missing_values = missing_values[missing_values > 0]

if not missing_values.empty:
    print("Columns with missing values:")
    
    for column, count in missing_values.items():
        print(f"{column}: {count} missing values")
        
else:
    print("There are no columns with missing value")

In [ ]:
# Check for duplicate data

if supply_data.duplicated().any():
    print(f"There are as many as {supply_data.duplicated().sum()} duplicate data.")
    
else:
    print("There are no duplicate data.")

In [ ]:
# Quality control check

defect_rates_by_product = supply_data.groupby("Product type")['Defect rates'].mean().reset_index()

# Create a bar chart using Plotly
fig = px.bar(defect_rates_by_product, x='Product type', y='Defect rates', title='Defect Rates by Product Type')

# Customize the color scale for bars
color_scale = px.colors.qualitative.Set3  # You can choose a different color scale
fig.update_traces(marker_color=color_scale)

# Customize the appearance of the chart
fig.update_layout(
    xaxis_title="Product Type",
    yaxis_title="Mean Defect Rates",
    xaxis=dict(categoryorder='total descending'),
    yaxis=dict(title='Mean Defect Rates'),
    plot_bgcolor='white',
    title_x=0.5,
    showlegend=True  # Hide the legend
)

# Show the plot
fig.show()

In [ ]:
# Supply chain risk management

risk_data = supply_data[['SKU', 'Lead times', 'Stock levels']]
risk_data.head()

In [ ]:
risk_data['Risk score'] = risk_data['Lead times'] * (1-risk_data.loc[:,'Stock levels'])

In [ ]:
# Sort the risk_data by 'Risk score' in descending order and select top 10 highest-risk data
risk_data = risk_data.sort_values(by='Risk score', ascending=False)[:10]

# Create a bar plot using Plotly Express
fig = px.bar(risk_data, x='SKU', y='Risk score', title='Top 10 Highest-Risk Data',
             labels={'Risk score': 'Risk Score', 'SKU': 'SKU'},
             text='Risk score')

# Customize the appearance of the plot
fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig.update_layout(xaxis_title='SKU', yaxis_title='Risk Score', title_x=0.5)

# Show the plot
fig.show()

In [1]:
# 

# we assume for holding cost 0.2
holdingcost = 0.2

In [ ]:
def calculate_eoq(data):
    S = data['Costs']
    D = data['Number of products sold'] 
    H = data['Number of products sold'] * holdingcost
    EOQ = np.sqrt((2*S*D)/H)
    return round(EOQ)

In [ ]:
supply_data['EOQ'] = calculate_eoq(supply_data)

In [ ]:
comparison_columns = supply_data[['SKU', 'EOQ','Order quantities']]
comparison_columns.head()

In [ ]:
# Extracting the top 10 rows for comparison
top_10_comparison = comparison_columns.head(10)

# Creating a bar plot to compare EOQ and Order Quantities for the top 10 SKUs
fig = px.bar(top_10_comparison, x='SKU', y=['EOQ', 'Order quantities'], 
             title='Comparison of EOQ and Order Quantities for Top 10 SKUs')

fig.update_layout(
    title_x=0.5
)

# Displaying the plot
fig.show()

In [ ]:
# Customer segmentation analysis

revenue_avg_by_demo_prod = supply_data.groupby(['Customer demographics', 'Product type'])['Revenue generated'].mean().reset_index()
revenue_sum_by_demo_prod = supply_data.groupby(['Customer demographics', 'Product type'])['Revenue generated'].sum().reset_index()

In [ ]:
# Create a palette of different colors for each product type
colors = px.colors.qualitative.Set3

# Create a subplot with 1 row and 2 columns
fig = make_subplots(rows=1, cols=2, subplot_titles=('Average Revenue', 'Total Revenue'))

# Plot for average revenue
for i, product_type in enumerate(revenue_avg_by_demo_prod['Product type'].unique()):
    subset = revenue_avg_by_demo_prod[revenue_avg_by_demo_prod['Product type'] == product_type]
    fig.add_trace(
        go.Bar(
            x=subset['Customer demographics'],
            y=subset['Revenue generated'],
            name=product_type,
            marker_color=colors[i]
        ),
        row=1, col=1
    )

# Plot for total revenue without legend
for i, product_type in enumerate(revenue_sum_by_demo_prod['Product type'].unique()):
    subset = revenue_sum_by_demo_prod[revenue_sum_by_demo_prod['Product type'] == product_type]
    fig.add_trace(
        go.Bar(
            x=subset['Customer demographics'],
            y=subset['Revenue generated'],
            showlegend=False,  # Hide legend for this subplot
            marker_color=colors[i]
        ),
        row=1, col=2
    )

# Beautify the subplot and show the legend for the first subplot
fig.update_layout(
    title='Revenue Analysis by Customer Demographics and Product Type',
    xaxis=dict(title='Customer Demographics'),
    yaxis=dict(title='Revenue'),
    xaxis2=dict(title='Customer Demographics'),
    yaxis2=dict(title='Revenue'),
    title_x=0.5,
    showlegend=True  # Show legend for the first subplot
)

# Show the subplot
fig.show()

In [ ]:
# Lead times optimization analysis

avg_lead_times_transport = supply_data.groupby(['Transportation modes'])['Lead times'].mean().reset_index()

fig = px.bar(avg_lead_times_transport, x='Transportation modes', y='Lead times',
             labels={'Transportation modes': 'Transportation Mode', 'Lead times': 'Average Lead Time'},
             title='Average Lead Times by Transportation Mode')

# Customize the color scale for bars
color_scale = px.colors.qualitative.Set3  # You can choose a different color scale
fig.update_traces(marker_color=color_scale)

# Customize the layout for beautification
fig.update_layout(
    title=dict(text='Average Lead Times by Transportation Mode', x=0.5),
    xaxis_title='Transportation Mode',
    yaxis_title='Average Lead Time',
    xaxis_tickangle=-45,  # Rotate x-axis labels for better readability
    font=dict(family="Arial", size=14),
    showlegend=False  # Remove legend
)

# Show the plot
fig.show()

In [ ]:
best_transportation_mode = avg_lead_times_transport.loc[avg_lead_times_transport['Lead times'].idxmin()]
best_transportation_mode

In [ ]:
best_transportation_data = supply_data[supply_data['Transportation modes']==best_transportation_mode['Transportation modes']]
best_transportation_data.head()

In [ ]:
avg_lead_times_route = supply_data.groupby(['Routes'])['Lead times'].mean().reset_index()

# Assuming you have the avg_lead_times_route DataFrame already defined
fig = px.bar(avg_lead_times_route, x='Routes', y='Lead times',
             labels={'Routes': 'Route', 'Lead times': 'Average Lead Time'},
             title='Average Lead Times by Route')

# Customize the color scale for bars
color_scale = px.colors.qualitative.Set3  # You can choose a different color scale
fig.update_traces(marker_color=color_scale)

# Customize the layout for beautification
fig.update_layout(
    title=dict(text='Average Lead Times by Route', x=0.5),
    xaxis_title='Route',
    yaxis_title='Average Lead Time',
    xaxis_tickangle=-45,  # Rotate x-axis labels for better readability
    font=dict(family="Arial", size=14),
    showlegend=False  # Remove legend
)

# Show the plot
fig.show()

In [ ]:
best_route = avg_lead_times_route.loc[avg_lead_times_route['Lead times'].idxmin()]
best_route

In [ ]:
best_route_data = supply_data[supply_data['Routes']==best_route['Routes']]
best_route_data.head()

In [ ]:
# Modelling

# Forecasting demand
X = supply_data.loc[:,['Price', 'Availability', 'Stock levels', 'Lead times', 'Order quantities']]
y = supply_data.loc[:,'Number of products sold']

In [ ]:
# Define the number of folds for cross-validation
num_folds = 10

In [ ]:
# Initialize lists to store evaluation metrics
mse_scores = []
rmse_scores = []
mae_scores = []
r2_scores = []

In [ ]:
# Create a KFold object
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

In [ ]:
# Define the parameters for LightGBM
params = {
    'objective': 'regression',
    'metric': 'mean_squared_error',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9
}

In [ ]:
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Create LightGBM datasets for training and testing
    train_data = lgb.Dataset(X_train, label=y_train)
    test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)
    
    # Add early stopping
    num_round = 100  # Increase the number of boosting rounds
    early_stopping_rounds = 5  # Set the number of rounds to wait for early stopping

    # Train the model with early stopping
    bst = lgb.train(
        params,
        train_data,
        num_round,
        valid_sets=[test_data],
        early_stopping_rounds=early_stopping_rounds,
        verbose_eval=10
    )
    
    # Make predictions on the test set
    y_pred = bst.predict(X_test, num_iteration=bst.best_iteration)

    # Calculate metrics
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    # Calculate the target range
    target_range = np.max(y_test) - np.min(y_test)

    # Calculate metrics as percentages
    percentage_mse = (mse / target_range) * 100
    percentage_rmse = (rmse / target_range) * 100
    percentage_mae = (mae / target_range) * 100
    percentage_r2 = (r2 * 100)

    # Append the scores to the respective lists
    mse_scores.append(percentage_mse)
    rmse_scores.append(percentage_rmse)
    mae_scores.append(percentage_mae)
    r2_scores.append(percentage_r2)

In [ ]:
# Calculate the average scores over all folds
avg_mse = np.mean(mse_scores)
avg_rmse = np.mean(rmse_scores)
avg_mae = np.mean(mae_scores)
avg_r2 = np.mean(r2_scores)

# Print the results
print(f"Average Mean Squared Error: {avg_mse:.2f}%")
print(f"Average Root Mean Squared Error: {avg_rmse:.2f}%")
print(f"Average Mean Absolute Error: {avg_mae:.2f}%")
print(f"Average R-squared: {avg_r2:.2f}%")

In [2]:
# Cost optimization

# Extract the features (X) and target (y)
X = supply_data.loc[:, 'Production volumes'].values.reshape(-1, 1)
y = supply_data.loc[:, 'Manufacturing costs'].values

In [ ]:
# Define the number of folds for cross-validation
num_folds = 5

In [ ]:
# Initialize lists to store evaluation metrics
mse_scores = []
rmse_scores = []
mae_scores = []
r2_scores = []

In [ ]:
# Initialize and fit the scaler to the data
scaler = MinMaxScaler()
scaler.fit(X)

In [ ]:
# Create a KFold object
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

In [ ]:
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Scale the data
    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Define and compile the neural network model
    model = tf.keras.Sequential([
        Dense(64, activation='relu', input_dim=1),
        Dense(32, activation='relu'),
        Dense(1)
    ])

    model.compile(optimizer='adam', loss='mean_squared_error')
    
    # Define Early Stopping callback
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    # Define Model Checkpoint callback to save the best model
    model_checkpoint = ModelCheckpoint('/kaggle/working/best_model.h5', save_best_only=True, save_weights_only=True, monitor='val_loss', mode='min')

    # Train the model on the training data with callbacks
    history = model.fit(X_train_scaled,
                        y_train, epochs=100,
                        batch_size=32,
                        validation_data=(X_test_scaled, y_test),
                        callbacks=[early_stopping, model_checkpoint])

    # Load the best model weights from the saved checkpoint
    model.load_weights('/kaggle/working/best_model.h5')
    
    # Make predictions on the test set
    y_pred = model.predict(X_test_scaled)
    
    # Calculate metrics
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = np.mean(np.abs(y_test - y_pred))
    r2 = r2_score(y_test, y_pred)

    # Calculate metrics as percentages
    target_range = np.max(y_test) - np.min(y_test)
    percentage_mse = (mse / target_range) * 100
    percentage_rmse = (rmse / target_range) * 100
    percentage_mae = (mae / target_range) * 100
    percentage_r2 = (r2 * 100)

    # Append the scores to the respective lists
    mse_scores.append(percentage_mse)
    rmse_scores.append(percentage_rmse)
    mae_scores.append(percentage_mae)
    r2_scores.append(percentage_r2)

In [ ]:
# Calculate the average scores over all folds
avg_mse = np.mean(mse_scores)
avg_rmse = np.mean(rmse_scores)
avg_mae = np.mean(mae_scores)
avg_r2 = np.mean(r2_scores)

# Print the results
print(f"Average Mean Squared Error: {avg_mse:.2f}%")
print(f"Average Root Mean Squared Error: {avg_rmse:.2f}%")
print(f"Average Mean Absolute Error: {avg_mae:.2f}%")
print(f"Average R-squared: {avg_r2:.2f}%")

In [ ]:
# Find the most optimal production volume to minimize manufacturing cost
min_production_volume = supply_data['Production volumes'].min()
max_production_volume = 1000
step_size = 100

cheapest_cost = float('inf')
best_production_volume = None

for production_volume in range(min_production_volume, max_production_volume + 1, step_size):
    normalized_production_volume = scaler.transform(np.array([[production_volume]]))
    predicted_cost = model.predict(normalized_production_volume)

    if predicted_cost[0][0] < cheapest_cost:
        cheapest_cost = predicted_cost[0][0]
        best_production_volume = production_volume

print('Most optimal production volume to minimize manufacturing cost:', best_production_volume)
print('The cheapest manufacturing cost:', cheapest_cost)